In [1]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import pandas as pd
from skimage import io, feature, color, measure, draw, img_as_float
import numpy as np
import csv
import random2
import statistics
import scipy
from scipy import signal
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import itertools
import os
from os import listdir
from os.path import isfile, join

## Define Prepocessing Function

In [2]:
#define parameters for HoughTransform outside of function to be able to save and manipulate easier
def hougdraw(submitted_image, dp = 1, mindist = 10000, param1 = 10, param2=22, minradius = 5, maxradius=250):
    circles = cv2.HoughCircles(submitted_image, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = mindist,  
                               param1 = param1, param2 = param2, 
                               minRadius = minradius, maxRadius = maxradius)
    if circles is not None:
        circles = np.round(circles[0, 0:1]).astype("int")
        circle1 = circles[0,0]
        circle2 = circles[0,1]
        circle3 = circles[0,2]
        for(x, y, r) in circles:
            cv2.circle(submitted_image, (x, y), r, (255, 255, 0), 2) #version without drawing roi back on whole image 
    return(submitted_image)

def preprocessing(image, medianblur = 27, 
                              dilation = 4, 
                              alpha = 1, 
                              beta= 15,
                              thresh_div_1=5,
                              thresh_div_2=10):
    #image0 = hougdraw(image)
    #convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #brightness change
    gray = cv2.convertScaleAbs(gray, alpha = alpha, beta = beta)
    #set dynamic tresholds for canny (and thus also for hough)
    mean_intensity = np.median(gray)
    threshold1 = int(max(0, (1.0 - 0.33) * mean_intensity/thresh_div_1))
    threshold2 = int(min(255, (1.0 + 0.33) * mean_intensity/thresh_div_2))    
    #blur
    image2 = cv2.medianBlur(gray, medianblur)
    #dynamic thresholds for canny edge detection based on intensity of image
    #Thresholds one standard deviation above and below median intensity
    #edge detection
    image3 = cv2.Canny(image2, threshold1, threshold2)
    #dilation and second blur
    submitted = cv2.dilate(image3, None, iterations= dilation)  
    image4 = cv2.medianBlur(submitted, medianblur) 
    #add hough
    image4 = np.float32(image4)
    return image4, threshold1, threshold2

# Loop to optimize parameters
## looping through manully tracked videos

In [3]:
#######################################
import random
#version 2, using videos #################### loading in the videos
videofolder = './snippets/videos/'
vids = [f for f in listdir(videofolder) if isfile(join(videofolder, f))]
vidlist = []

for i in vids: #add the image folder name to get the full path
    vidlist.append(videofolder +i) 

#######################################
for video in vidlist:
    name = os.path.basename(video)[0:-4]
    #set up empty output dataframe
    column_names = ['frame', # info on region of interest for repetability
                            'x','y', 'r', 'name', 'sample_rate']# parameters of hough circle transform 
    df = pd.DataFrame(columns = column_names)
    roi = [0, 0, 0, 0]
            
    ####################DO ROUTINEand 
    cap = cv2.VideoCapture(video)
    frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second
    #also set up a video to track
        #make an 'empty' video file where we project the pose tracking on
    fourcc = cv2.VideoWriter_fourcc(*'MP4V') #for different video formats you could use e.g., *'XVID'
    out = cv2.VideoWriter('snippets/tracked_hough/' + name + '_tracked.mp4', fourcc, 
                          fps = fps, frameSize = (int(frameWidth), int(frameHeight)))
    
    #for imtoprocess in videos:
    j = 0   
    while(cap.isOpened()):
        ret, frame = cap.read()
        j=j+1

        if ret == False:
            break
        ############################detect circles   
        output=frame.copy()
        #image4, param1, param2 = preprocessing(image=output)
        image4, param1, param2 = preprocessing(image=output)
            #track demicircles 
        name = 'framenr_' + str(j) + '_framevid_' + os.path.basename(video[0:-4])
        #process image for hough
        final_im = cv2.normalize(src=image4, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) 
        circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT,param1 = param1,param2 = param2, dp = 1, minDist = 10000, maxRadius = 250)   
       
        if circles is not None:
            if circles is not None:
                circles = np.round(circles[0, 0:1]).astype("int")
                circle1 = circles[0,0] #x  + plus the shift from the roi
                circle2 = circles[0,1] #y  + plus the shift from the roi
                circle3 = circles[0,2]
            #    #save it to a row
                for(x, y, r) in circles:
                    cv2.circle(output, (x, y), r, (255, 255, 0), 2) #version without drawing roi back on whole image 
            if circles is None:
                circle1 = "NA"
                circle2 = "NA"
                circle3 = "NA"
        out.write(output) #save the frame to the new masked video
        new_row = [j, circle1, circle2,circle3, name, fps]
        df.loc[len(df)] = new_row
    out.release()
    df.to_csv('./snippets/tracked_hough/'+name+'.csv', sep = ',')
print('done!')


done!


# Loop through all videos in the video folder